In [1]:
import numpy as np
import pandas as pd
import pickle
from keras.datasets import cifar10

2023-07-27 10:10:08.748088: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 10:10:08.782126: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 10:10:09.277009: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load training and testing data separately
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [3]:
# Reshape x_train from 4D to 2D array (number of samples, width*height*channels)
x_train = x_train.reshape(x_train.shape[0], -1)

# Reshape y_train to 1D array
y_train = y_train.reshape(-1)

# Combine training data and labels into a single numpy array for easier manipulation
train_data = np.column_stack((x_train, y_train))

# Randomly shuffle the training data
np.random.shuffle(train_data)


In [4]:
def get_chunks_with_overlap(data, num_chunks=10, min_chunk_size=0.2, max_chunk_size=0.4):
    num_samples = len(data)
    num_classes = 10  # cifar10 has 10 classes

    # Select random chunk sizes that are multiples of the number of classes
    chunk_sizes = [np.random.choice(np.arange(int(min_chunk_size * num_samples), 
                                              int(max_chunk_size * num_samples)+1, 
                                              num_classes)) for _ in range(num_chunks)]
    
    chunks = []
    
    # Split data by classes
    data_by_class = {i: [] for i in range(num_classes)}
    for row in data:
        data_by_class[int(row[-1])].append(row)

    # Ensure each sample is included at least once
    for i in range(num_classes):
        np.random.shuffle(data_by_class[i])

    # Copy of the original class samples before popping
    data_by_class_original = {i: list(data) for i, data in data_by_class.items()}
    
    for size in chunk_sizes:
        chunk = []
        for _ in range(size // num_classes):
            for c in range(num_classes):
                if data_by_class[c]:  # if there are samples left
                    chunk.append(data_by_class[c].pop())
                else:  # if all samples of this class have been used, start reusing
                    chunk.append(data_by_class_original[c][np.random.choice(len(data_by_class_original[c]))])
        chunks.append(np.array(chunk))
    return chunks

# Split train_data into 10 chunks
chunks = get_chunks_with_overlap(train_data)


In [5]:
def remove_duplicate_sample_from_chunk(chunks):
    unique_chunks = []
    additional_samples = []
    num_classes = 10  # cifar10 has 10 classes

    for i, chunk in enumerate(chunks):
        unique_samples = np.unique(chunk, axis=0)
        unique_chunks.append(unique_samples)

        # Count the number of samples from each category in the original and unique chunks
        original_counts = np.bincount(chunk[:,-1], minlength=num_classes)
        unique_counts = np.bincount(unique_samples[:,-1], minlength=num_classes)

        # Compute the additional_samples for each category as the difference between the original and unique counts
        additional_sample = original_counts - unique_counts
        additional_samples.append(additional_sample.tolist())
        print(f"chunk {i+1}: {len(chunk)} vs {len(unique_samples)} --> additional sample = {additional_sample.tolist()}")
        print(f"additional sample list: {additional_sample}")
        
    return zip(unique_chunks, additional_samples)

requirement = remove_duplicate_sample_from_chunk(chunks)


chunk 1: 15380 vs 15380 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 2: 17450 vs 17450 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 3: 13990 vs 13990 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 4: 10570 vs 9615 --> additional sample = [106, 88, 97, 90, 101, 94, 99, 107, 88, 85]
additional sample list: [106  88  97  90 101  94  99 107  88  85]
chunk 5: 18220 vs 15208 --> additional sample = [309, 292, 303, 304, 302, 297, 307, 307, 282, 309]
additional sample list: [309 292 303 304 302 297 307 307 282 309]
chunk 6: 17190 vs 14469 --> additional sample = [281, 275, 259, 270, 240, 263, 282, 272, 295, 284]
additional sample list: [281 275 259 270 240 263 282 272 295 284]
chunk 7: 19530 vs 16079 --> additional sample = [345, 332, 340, 351, 343, 330, 362, 336, 362, 350]
additional sample list: [345 332 340 35

In [6]:
def fill_chunks(requirement, chunks, subset_factor=2):
    num_classes = 10  # cifar10 has 10 classes

    for i, (unique_chunk, additional_samples) in enumerate(requirement):
        # Create a set for faster membership tests
        unique_set = set(map(tuple, unique_chunk))

        # Convert the list of additional samples into a numpy array for easier manipulation
        additional_samples = np.array(additional_samples)

        while np.any(additional_samples > 0):
            # Randomly select a chunk different from the current one
            while True:
                random_chunk_index = np.random.choice(len(chunks))
                if random_chunk_index != i:
                    break

            random_chunk = chunks[random_chunk_index]
            np.random.shuffle(random_chunk)

            for sample in random_chunk:
                # If sample is not already in unique_set and there are still missing samples for its category
                if tuple(sample) not in unique_set and additional_samples[int(sample[-1])] > 0:
                    # Add sample to unique_chunk
                    unique_chunk = np.vstack([unique_chunk, sample])
                    # Add sample to unique_set
                    unique_set.add(tuple(sample))
                    # Decrease the count of missing samples for this category
                    additional_samples[int(sample[-1])] -= 1

            # Replace the old chunk with the new one in the chunks list
            chunks[i] = unique_chunk

    return chunks

# Use the fill_chunks function
filled_chunks = fill_chunks(requirement, chunks)


In [7]:
result = remove_duplicate_sample_from_chunk(filled_chunks)

chunk 1: 15380 vs 15380 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 2: 17450 vs 17450 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 3: 13990 vs 13990 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 4: 10570 vs 10570 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 5: 18220 vs 18220 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 6: 17190 vs 17190 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 7: 19530 vs 19530 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]
chunk 8: 19380 vs 19380 --> additional sample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
additional sample list: [0 0 0 0 0 0 0 0 0 0]


In [8]:
# Unzip the result to get chunks and additional_samples separately
chunks, _ = zip(*result)

for chunk in chunks:
    print(chunk.shape)



(15380, 3073)
(17450, 3073)
(13990, 3073)
(10570, 3073)
(18220, 3073)
(17190, 3073)
(19530, 3073)
(19380, 3073)
(13620, 3073)
(10080, 3073)


In [9]:

# Function to get label distribution in a chunk
def get_label_distribution(chunk):
    # The label is in the last column
    labels = chunk[:, -1]
    unique_labels, counts = np.unique(labels, return_counts=True)
    return dict(zip(unique_labels, counts))


# Function to get label proportions in a chunk
def get_label_proportions(label_distribution, chunk_size):
    proportions = {}
    for label, count in label_distribution.items():
        proportions[label] = count / chunk_size
    return proportions




In [10]:
import os
folder = "10_chunks_overlap"
if not os.path.exists(folder):
    os.makedirs(folder)


In [11]:
iid_folder = "iid"
iid_folder = os.path.join(folder, iid_folder)
if not os.path.exists(iid_folder):
    os.makedirs(iid_folder)


In [12]:
# iid distribution

chunk_info = []
for i, chunk in enumerate(chunks):

    label_distribution = get_label_distribution(chunk)
    chunk_size =len(chunk)
    # save info
    info = {}
    info['chunk'] = i+1
    info['size'] = chunk_size
    info['label_distribution'] = label_distribution
    proportions = get_label_proportions(label_distribution, chunk_size)
    info['label_proportions'] = proportions

    chunk_info.append(info)

    # Save unbalanced chunk as a pickle file
    with open(f'{iid_folder}/chunk_{i+1}.pickle', 'wb') as f:
        pickle.dump(chunk, f)


In [13]:
# Convert list of dictionaries to DataFrame for better visualization
df = pd.DataFrame(chunk_info)

# print dataframe
print(df)



   chunk   size                                 label_distribution  \
0      1  15380  {0: 1538, 1: 1538, 2: 1538, 3: 1538, 4: 1538, ...   
1      2  17450  {0: 1745, 1: 1745, 2: 1745, 3: 1745, 4: 1745, ...   
2      3  13990  {0: 1399, 1: 1399, 2: 1399, 3: 1399, 4: 1399, ...   
3      4  10570  {0: 1057, 1: 1057, 2: 1057, 3: 1057, 4: 1057, ...   
4      5  18220  {0: 1822, 1: 1822, 2: 1822, 3: 1822, 4: 1822, ...   
5      6  17190  {0: 1719, 1: 1719, 2: 1719, 3: 1719, 4: 1719, ...   
6      7  19530  {0: 1953, 1: 1953, 2: 1953, 3: 1953, 4: 1953, ...   
7      8  19380  {0: 1938, 1: 1938, 2: 1938, 3: 1938, 4: 1938, ...   
8      9  13620  {0: 1362, 1: 1362, 2: 1362, 3: 1362, 4: 1362, ...   
9     10  10080  {0: 1008, 1: 1008, 2: 1008, 3: 1008, 4: 1008, ...   

                                   label_proportions  
0  {0: 0.1, 1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0....  
1  {0: 0.1, 1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0....  
2  {0: 0.1, 1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0....  
3  {0: 0.

In [14]:
# Save dataframe to csv
df.to_csv(f"{iid_folder}/chunks_info.csv", index=False)
